What I did: 
First Accuracy: 0.00
Why: because relu_Predict_all_all_all_all_all didn't have values 0 or 1 only intermediate
    Change - ran a for loop to make all values 0 or 1
Second Accuracy: .12897
Why: Possibly because I miss labeled values 0 and 1
    Change - I swapped the values of 0 and 1
Third Accuracy: .879129
Why: All values in the test data are positive?
    Change - Mistake in the changing from 2-16 to 1 Also swapped label values back to normal - reverted second trial
Fourth Accuracy: 0.3846154 Also tested against only types 1 and 2 and got .14035
Why: ? possibly poor feature selection? still looking into it -- Standardize the data first with sklearn standardscalar 

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


df = pd.read_csv("./data_arrhythmia.csv", delimiter=";")

In [124]:
#Some of the columns have only one value so we will classify them as no info
no_info_cols = []
num_cols = df.shape[1]
for index in range(num_cols):
    if len(df.iloc[:,index].unique()) == 1:
        no_info_cols.append(df.columns[index])
no_info_cols

["S'_wave",
 'CB',
 'CD',
 'CS',
 'EV',
 'EY',
 'FF',
 'FH',
 'FJ',
 'FL',
 'FS',
 'FZ',
 'GA',
 'GH',
 'IB',
 'KP',
 'LC']

In [125]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
#height seems to be wrong, and age 0 and weight 10kg suggest missing decimal point for 60.8
#changing value of height to 61 cm
df.loc[df['age'] == 0, 'height'] = 61
#age 1 weight 6 suggest height to be 78 cm rather than 780
df.loc[df['height'] == 780, 'height'] = 78
df1 = df.copy()
df1 = df.drop(columns=no_info_cols)
df2 = df1.copy()
df2 = df1.drop(columns=['J'])
#wasn't registering ? as NaN had to replace
df2 = df2.replace('?', np.nan)
#changing all of the NaN to median values
df3 = df2.copy()
X_df = df3.loc[:,df3.columns != 'diagnosis']
X = np.array(X_df)
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp = imp.fit(X)
X_imp = imp.transform(X)
X_imp = pd.DataFrame(X_imp)
y = df2[df2.columns[-1]]
X_train_with_all, X_test_with_all, y_train_with_all, y_test_with_all = train_test_split(X_imp, y, test_size=.2, random_state = 0)
type_1 = set(y_train_with_all[y_train_with_all == 1].index)
type_2 = set(y_train_with_all[y_train_with_all != 2].index)
type_1_2 = type_2 - type_1
X_train_without_all = X_train_with_all.copy()
X_train_without_all = X_train_with_all.drop(type_1_2)
y_train_without_all = y_train_with_all.copy
y_train_without_all = y_train_with_all.drop(type_1_2)

type_1 = set(y_test_with_all[y_test_with_all == 1].index)
type_2 = set(y_test_with_all[y_test_with_all != 2].index)
type_1_2 = type_2 - type_1
X_test_without_all = X_test_with_all.copy
X_test_without_all = X_test_with_all.drop(type_1_2)
y_test_without_all = y_test_with_all.copy()
y_test_without_all = y_test_with_all.drop(type_1_2)


In [126]:
from sklearn import preprocessing

X_train_without_all = preprocessing.StandardScaler().fit_transform(X_train_without_all)
X_test_with_all = preprocessing.StandardScaler().fit_transform(X_test_with_all)
X_test_without_all = preprocessing.StandardScaler().fit_transform(X_test_without_all)


In [127]:
#IMPLEMENT FISHERS SCORE FOR FEATURE SELECTION

In [128]:
#Changed values from 1 is negative and 2 is positive to 0 neg 1 pos
y_train_without_all = y_train_without_all.replace(1,0)
y_train_without_all = y_train_without_all.replace(2,1)


In [129]:
#started with 25-15-10 relu, added 1 sigmoid because binary_crossentropy requires it to be functional
import tensorflow as tf
import keras
from keras import layers

tf.random.set_seed(1234)
model_Relu = keras.Sequential(
    [
        layers.Dense(units=25, activation="relu", kernel_initializer='normal', name="layer1"),
        layers.Dense(units=15, activation="relu", kernel_initializer='normal', name="layer2"),
        layers.Dense(units=10, activation="relu", kernel_initializer='normal', name="layer3"),
        layers.Dense(units=1, activation="sigmoid", kernel_initializer='normal', name="layer4"),
    ]
)
model_Relu.compile(loss='binary_crossentropy')
model_Relu.fit(X_train_without_all, y_train_without_all)
relu_Predict_all = model_Relu.predict(X_test_with_all)
relu_Predict_without_all = model_Relu.predict(X_test_without_all)

2/2 [==============================] - 0s 8ms/step


In [139]:
def bump_activation(x, sigma=0.5):
   
    return tf.math.exp(-0.5 * tf.math.square(x) / tf.math.square(sigma))

model_Bump = keras.Sequential(
    [
        layers.Dense(units=25, activation=lambda x: bump_activation(x), kernel_initializer='normal', name="layer1"),
        layers.Dense(units=15, activation=lambda x: bump_activation(x), kernel_initializer='normal', name="layer2"),
        layers.Dense(units=10, activation=lambda x: bump_activation(x), kernel_initializer='normal', name="layer3"),
        layers.Dense(units=1, activation="sigmoid", kernel_initializer='normal', name="layer4"),
    ]
)
model_Bump.compile(loss='binary_crossentropy')
model_Bump.fit(X_train_without_all, y_train_without_all)
bump_Predict_all = model_Bump.predict(X_test_with_all)
bump_Predict_without_all = model_Bump.predict(X_test_without_all)

2/2 [==============================] - 0s 9ms/step


In [131]:
def get_metrics(test_values, predict_values):
    accuracyF = metrics.Accuracy()
    accuracyF.update_state(test_values, predict_values)
    accuracy = accuracyF.result().numpy()
    print("Accuracy: " + str(accuracy))

    truePositiveF = metrics.TruePositives()
    truePositiveF.update_state(test_values, predict_values)
    truePositive = truePositiveF.result().numpy()
    print("True Positive: " +  str(truePositive))

    trueNegativeF = metrics.TrueNegatives()
    trueNegativeF.update_state(test_values, predict_values)
    trueNegative = trueNegativeF.result().numpy()
    print("True Negative: " +  str(trueNegative))

    falsePositiveF = metrics.FalsePositives()
    falsePositiveF.update_state(test_values, predict_values)
    falsePositive = falsePositiveF.result().numpy()
    print("False Positive: " +  str(falsePositive))

    falseNegativeF = metrics.FalseNegatives()
    falseNegativeF.update_state(test_values, predict_values)
    falseNegative = falseNegativeF.result().numpy()
    print("False Negative: " +  str(falseNegative))

In [132]:
y_test_with_all_value_0_1 = y_test_with_all
y_test_with_all_value_0_1 = y_test_with_all.replace(1,0)
y_test_with_all_value_0_1 = y_test_with_all_value_0_1.replace([2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
y_test_with_all_value_0_1.to_numpy()

y_test_with_out_all_value_0_1 = y_test_without_all
y_test_with_out_all_value_0_1 = y_test_without_all.replace(1,0)
y_test_with_out_all_value_0_1 = y_test_with_out_all_value_0_1.replace(2,1)
y_test_with_out_all_value_0_1.to_numpy()

array([0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [133]:
# converted all values of relu_Predict_all to either 0 or 1
for i in range(len(relu_Predict_all)):
    if relu_Predict_all[i]< .5:
        relu_Predict_all[i] = 1
    else:
        relu_Predict_all[i] = 0

for i in range(len(relu_Predict_without_all)):
    if relu_Predict_without_all[i]< .5:
        relu_Predict_without_all[i] = 1
    else:
        relu_Predict_without_all[i] = 0


In [134]:
from tensorflow import metrics
get_metrics(y_test_with_all_value_0_1, relu_Predict_all)

Accuracy: 0.46153846
True Positive: 42.0
True Negative: 0.0
False Positive: 49.0
False Negative: 0.0


In [135]:
get_metrics(y_test_with_out_all_value_0_1, relu_Predict_without_all)


Accuracy: 0.14035088
True Positive: 8.0
True Negative: 0.0
False Positive: 49.0
False Negative: 0.0


In [141]:
get_metrics(y_test_with_all, bump_Predict_all)

Accuracy: 0.0
True Positive: 0.0
True Negative: 0.0
False Positive: 0.0
False Negative: 91.0


In [140]:
get_metrics(y_test_without_all, bump_Predict_without_all)

Accuracy: 0.0
True Positive: 0.0
True Negative: 0.0
False Positive: 0.0
False Negative: 57.0
